# Modele językowe - n-gramy 

---

## 1. N-gramy słów w klasyfikacji
Poniżej stworzono kod, który przeprowadza klasyfikację dokumentów należących do 4 kategorii. W odróżnieniu do poprzednich zajęć - tu zaproponowano klasyfikator SVC (algorytm SVM, popularna alternatywa dla NaiveBayes), która również świetnie się spisuje w problemach klasyfikacji tekstu.

**<span style="color: red">Zadanie 1a (0.5 punktu)</span>** Uruchom kod, przyjrzyj się wygenerowanym wynikom (a najlepiej zachowaj je gdzieś, będą potrzebne). <br/>
Zapoznaj się z dokumentacją TfIdfVectorizer, odnajdź opcję uwzględnienia nie tylko pojedynczych słów jako cechy, ale także ich par i **zmodyfikuj poniższy kod tak, aby klasyfikacja uwzględniała zarówno pojedyncze słowa jak i pary (pozostaw parametr max_df=0.1 nienaruszony).** <span style="color: red"> Zmodyfikuj linię 30.</span><br/> <br/>
**<span style="color: red">Zadanie 1b (0.5 punktu)</span>** Jak zmieniła się liczba cech po uwzględnieniu tych par? Czy coś się zmieniło w raporcie z klasyfikacji? Uzupełnij odpowiedzi na pytania w komórce poniżej kodu.

W słowniku znajduje się 34774 różnych cech
              precision    recall  f1-score   support

           0       0.88      0.93      0.90       389
           1       0.91      0.93      0.92       385
           2       0.94      0.91      0.92       394
           3       0.96      0.90      0.93       310

    accuracy                           0.92      1478
   macro avg       0.92      0.92      0.92      1478
weighted avg       0.92      0.92      0.92      1478

W słowniku znajduje się 287718 różnych cech
              precision    recall  f1-score   support

           0       0.87      0.94      0.91       389
           1       0.92      0.94      0.93       385
           2       0.95      0.90      0.92       394
           3       0.97      0.90      0.93       310

    accuracy                           0.92      1478
   macro avg       0.93      0.92      0.92      1478
weighted avg       0.92      0.92      0.92      1478

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.datasets import fetch_20newsgroups # zbiór danych zawarty w Sklearn, który zawiera dane z 20 grup newsowych
import numpy as np

# ------------------- WCZYTANIE DANYCH -----------

np.random.seed(0) # ustaw seed na 0, aby zapewnić powtarzalność eksperymentu

categories = ['sci.space', 'comp.graphics', 'talk.politics.misc', 'comp.sys.mac.hardware'] # kategorie do analizy

train = fetch_20newsgroups(subset='train',
                                   categories=categories,
                                   shuffle=True,
                                   random_state=42) # pobieramy zbiór uczący (na nim będziemy trenować) dla wybranych kategorii.
    

test = fetch_20newsgroups(subset='test',
                                  categories=categories,
                                  shuffle=True,
                                  random_state=42) # pobieramy zbiór testowy (na nim będziemy testować) dla wybranych kategorii

# ------------------- STWORZENIE PIPELINE'U -----------
    
pipeline = Pipeline([             # stwórzmy pipeline surowy tekst -> TFIDF vectorizer -> klasyfikator 
    ('tfidf', TfidfVectorizer(ngram_range=(1,2), max_df=0.1)),
    ('clf', SVC(C=1.0, kernel='linear')),
])

# ------------------- TRANSFORMACJA I UCZENIE -----------

pipeline.fit(train.data, train.target) # zwektoryzujmy dane i wytrenujmy klasyfikator na zbiorze treningowym


print("W słowniku znajduje się {n} różnych cech".format(
    n=len(pipeline.named_steps['tfidf'].vocabulary_.keys())
))

# ------------------- OCENA KLASYFIKATORA -----------
print(classification_report(test.target, pipeline.predict(test.data))) # testowanie klasyfikatora - szerokie podsumowanie uwzględniające miary: precision, recall, f1

W słowniku znajduje się 287718 różnych cech
              precision    recall  f1-score   support

           0       0.87      0.94      0.91       389
           1       0.92      0.94      0.93       385
           2       0.95      0.90      0.92       394
           3       0.97      0.90      0.93       310

    accuracy                           0.92      1478
   macro avg       0.93      0.92      0.92      1478
weighted avg       0.92      0.92      0.92      1478



1. O ile zwiększyła się liczba cech w klasyfikatorze? Wzrosła z 34_774 do 287_718, czyli o 25_944
2. Czy precyzja w którejkolwiek klasie wzrosła? w której/których? Tak, wzrosła w klasach 1, 2, 3
3. Czy recall w którejkolwiek klasie wzrósł? w której/których? Tak, wzrósł w klasach 0, 1

## 2. N-gramy liter w klasyfikacji
Poza n-gramami stworzonymi z następujących po sobie wyrazów - bardzo często używane są również n-gramy znakowe, stworzone z następujących po sobie liter. <br/><br/>
Dla przykładu. wszystkie 3-gramy (trigramy) znakowe z napisu "Hello world" to: <br/>
"Hel", "ell", "llo", "lo ", "o w", " wo", "wor", "orl", "rld". <br/><br/>
Do czego mogłaby być użyta taka reprezentacja tekstów? Okazuje się, że całkiem mocno pomaga to w rozwiązywaniiu problemu detekcji języka w którym został zapisany dokument, szczególnie w sytuacji, kiedy teksty są bardzo krótkie (np. tweety, smsy).
<br/>
Poniżej znajduje się szkielet klasyfikatora rozpoznającego język w którym zapisany jest dokument.
Języków jest 6: polski, angielski, niemiecki, francuski, hiszpański i włoski.
<br/>
**<span style="color: red">Zadanie 2 (1 punkt)</span>**: Przedstawiony klasyfikator jest znanym już z poprzednich przykładów kodem. Waszym zadaniem jest:
<ol>
    <li>Zapoznanie się dokumentacją Tf-Idf vectorizera, aby znaleźć funkcjonalność, która zamiast całych słów, stworzy cechy na podstawie liter i wykorzystanie tej funkcjonalności w kodzie</li>
    <li>Ustawienie takiego zakresu n-gramów, aby zmaksymalizować uzyskany wynik (Oczekiwane 1.0 precyzji i recallu we wszystkich kategoriach przy pozostawieniu wartośi max_features = 300 elementów)</li>
    <li>Poprawnie zaklasyfikuje krotki przykład zapisany w linii 43 (Bonjour przypisze do kategorii 'french').</li>
</ol>

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas
import numpy as np

# -------------------- FUNKCJE POMOCNICZNE --------

# Funkcja mapująca identyfikator liczbowy kategorii na wartość tekstową, np: 0->"polish", 1->"english"
def get_class_name_from_id(ids, mapping):
    return [mapping[id] for id in ids]

# ------------------- WCZYTANIE DANYCH -----------

full_dataset = pandas.read_csv('language_detection_1000.csv', encoding='utf-8') # wczytaj dane z pliku CSV
lang_to_id = {'polish': 0, 'english': 1, 'french': 2,
              'german': 3, 'italian': 4, 'spanish': 5}
id_to_lang = {v: k for k,v in lang_to_id.items()}

full_dataset['label_num'] = full_dataset.lang.map(lang_to_id)  # ponieważ nazwy kategorii zapisane są z użyciem stringów: "ham"/"spam", wykonujemy mapowanie tych wartości na liczby, aby móc wykonać klasyfikację. 

np.random.seed(0)                                       # ustaw seed na 0, aby zapewnić powtarzalność eksperymentu
train_indices = np.random.rand(len(full_dataset)) < 0.7 # wylosuj 70% wierszy, które znajdą się w zbiorze treningowym

train = full_dataset[train_indices] # wybierz zbior treningowy (70%)
test = full_dataset[~train_indices] # wybierz zbiór testowy (dopełnienie treningowego - 30%)


# ------------------- STWORZENIE PIPELINE'U -----------
    
pipeline = Pipeline([             # stwórzmy pipeline surowy tekst -> TFIDF vectorizer -> klasyfikator 
    ('tfidf', TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4), max_features=300)),
    ('scaler', StandardScaler(with_mean = False)),
    ('clf', LogisticRegression()),
])

# ------------------- TRANSFORMACJA I UCZENIE -----------

pipeline.fit(train['text'], train['label_num']) # zwektoryzujmy dane i wytrenujmy klasyfikator na zbiorze treningowym


print("Oto kilka przykładowych cech stworzonych przez TfidfVectorizer: {n}".format(
    n=list(pipeline.named_steps['tfidf'].vocabulary_.keys())[:5]))

# ------------------- WERYFIKACJA NA KRÓTKIM TEKŚCIE ----

text_to_predict = "Bonjour!"
predicted = pipeline.predict([text_to_predict])
print("\n\nTekst: {t} został zaklasyfikowany jako: {p}\n\n".format(
    t=text_to_predict,
    p=id_to_lang[predicted[0]]
))


# ------------------- OCENA KLASYFIKATORA -----------
print(classification_report(
    get_class_name_from_id(test['label_num'], id_to_lang), 
    get_class_name_from_id(pipeline.predict(test['text']), id_to_lang)
))

/home/max/.local/opt/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/max/.local/opt/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Oto kilka przykładowych cech stworzonych przez TfidfVectorizer: ['ap', 'pe', 'el', 'lu', 'je']


Tekst: Bonjour! został zaklasyfikowany jako: french


              precision    recall  f1-score   support

     english       1.00      1.00      1.00       303
      french       1.00      1.00      1.00       280
      german       1.00      1.00      1.00       337
     italian       1.00      1.00      1.00       273
      polish       1.00      1.00      1.00       291
     spanish       1.00      1.00      1.00       299

    accuracy                           1.00      1783
   macro avg       1.00      1.00      1.00      1783
weighted avg       1.00      1.00      1.00      1783



Widzimy, że problem jest stosunkowo prosty. Po co zatem używać n-gramów znakowych? Aby zaoszczędzić pamięć i podołać sytuacjom, w których zbiór testowy składa się ze słów, które nie występują w korpusie uczącym. <br/>

O ile wszystkich słów w tych 6 językach jest "30078", to trigramów znakowych jest już tylko "15274", a bigramów - "2059". W związku z tym: <ol>
<li>Używając n-gramów znakowych często możemy ograniczyć liczbę cech</li>
<li>N-gramy znakowe pomogą nam w sytuacjach, kiedy dane słowo nie wystąpiło w tekście uczącym. Jeśli opieramy uczenie na pełnych słowach i cały nasz tekst testowy to niewystępujące w korpusie uczącym - "bonjour", wtedy wektor BOW będzie zawierał same zera, przez co będzie miał problem z przydziałem do odpowiedniej klasy. <br/> N-gramy znakowe nawet jeśli nie napotkały danego słowa podczas analizy korpusu, to na podstawie budowy samego słowa są w stanie przewidywać do jakiego języka słowo należy. Np. cokolwiek zawierającego trigram "żeb" należeć będzie raczej do języka polskiego.</li>
</ol>

---

### 2a - istotność cech

Ponieważ w zadaniu 2 użyliśmy znanego z zajęć z przedmiotu "Sztuczna Inteligencja" klasyfikatora liniowego - regresji logistycznej, podejrzeć możemy jakie cechy najsilniej sugerują nam przynależność do danej klasy. Uruchom poniższy kod, aby zobaczyć jakie cechy są najważniejsze dla danych kategorii. Modyfikując parametry TfidfVectorizer możesz zobaczyć jakie słowa/ciągi znaków są najistotniejsze do detekcji danego języka.

In [7]:
# Funkcja, z użyciem której możemy ocenić które cechy najsilniej skojarzone są z danymi klasami.
# Wyświetli listę słów/n-gramów znakowych, których obecność najsilniej wpływa na przydział do danej klasy
def language_indicators(feature_names, feature_importances, id_to_lang):
    for i, language in enumerate(feature_importances): # iterujemy po macierzy feature_importances (wymiarów: język x cechy) wierszami (czyli język po języku)
        scored_features = list(zip(feature_names, language)) # tworzymy skojarzenie nazw cech z wagami modelu (ponieważ używamy regresji logistycznej - każda cecha (słowo/n-gram) ma swoją wagę, która jest optymalizowana w procesie uczenia! Cechy z wysokimi wagami są ważne dla danej klasy. Każda klasa ma osobny model ze swoimi wagami!)
        scored_features = sorted(scored_features, key=lambda x: x[1], reverse=True) # posortujmy cechy skojarzone z wagami malejąco 
        print("W rozpoznaniu języka {lang} najważniejsze cechy to:".format(
            lang=id_to_lang[i]) #zamieńmy identyfikator numeryczny kategorii na nazwę języka
        )
        for feature, score in scored_features[:5]: # wybierzmy po 5 najważniejszych cech (najwyższe wartości uczonych współczynników)
            print("\t'{feature}': {score}".format(feature=feature, score=score))
        

# ------------------- WYŚWIETLENIE NAJWAŻNIEJSZYCH CECH DLA KAŻDEJ KATEGORII
language_indicators(
    pipeline.named_steps['tfidf'].get_feature_names(), # pobierz nazwy cech
    pipeline.named_steps['clf'].coef_, # pobierz wyuczone współczynniki (regresja logistyczna to stworzenie modelu opisanego wzorem y = e^(-wx - b), gdzie uczymy się współczynników w. Pole coef_ zawiera te współczynniki dla każdego języka z osobna)
    id_to_lang # mapowanie z identyfikatora numerycznego na pełną nazwę języka - zwiększa czytelność wygenerowanego raportu
)

W rozpoznaniu języka polish najważniejsze cechy to:
	'cz': 0.3043394460519647
	'ze': 0.2801719411323314
	'ej': 0.2671480744004373
	'rz': 0.26543384326299235
	'zy': 0.2506887018157753
W rozpoznaniu języka english najważniejsze cechy to:
	'th': 0.4906105190721736
	' th': 0.4487101451811298
	'the': 0.41083508951592124
	' an': 0.40080627205302904
	' the': 0.388489331399117
W rozpoznaniu języka french najważniejsze cechy to:
	'ce': 0.4999542235691582
	'ou': 0.46044639331080034
	't ': 0.3857235607313067
	'ai': 0.3777508555962045
	'ns': 0.3360591080187681
W rozpoznaniu języka german najważniejsze cechy to:
	'ei': 0.3904458576260525
	'die': 0.3633768546491568
	'en ': 0.3584113558878921
	'der': 0.33423718857621404
	'en': 0.32042206990550626
W rozpoznaniu języka italian najważniejsze cechy to:
	'i ': 0.5842019009936263
	'o ': 0.4799359807024486
	'tt': 0.3763116446558284
	'ss': 0.33959874151276037
	'ione': 0.3318918156530659
W rozpoznaniu języka spanish najważniejsze cechy to:
	'os ': 0.533474815

---

## 3. N-gramy słów w generowaniu tekstu

Innym, bardzo ciekawym zastosowaniem n-gramów jest możliwość generowania tekstu z użyciem tzw. łańcuchów Markova. Stwórzmy funkcję generującą n-gramy słów, aby później móc ją wykorzystać do tworzenia tekstów.

**<span style="color: red">Zadanie 3 (1 punkt)</span>** stwórz funkcję, która wygeneruje n-gramy słów zadanego stopnia n (n_gram_len). Aby podzielić zdanie na słowa nie musisz używać tokenizatora z biblioteki, na potrzeby zadania wystarczy uznać, że spacja oddziela poszczególne słowa.
<br/>
<br/>
<div class="alert alert-success">
Oczekiwany rezultat dla zadanych danych: <br/><br/>[['The', 'big', 'brown'], ['big', 'brown', 'fox'], ['brown', 'fox', 'jumped'], ['fox', 'jumped', 'over'], ['jumped', 'over', 'the'], ['over', 'the', 'fence.']]
</div>
<br/>

In [11]:
def get_word_ngrams(data, n_gram_len):
    tokens = data.split(' ')
    return [tokens[i:i+n_gram_len] for i in range(len(tokens) - n_gram_len + 1)]
print(get_word_ngrams("The big brown fox jumped over the fence.", 3))

[['The', 'big', 'brown'], ['big', 'brown', 'fox'], ['brown', 'fox', 'jumped'], ['fox', 'jumped', 'over'], ['jumped', 'over', 'the'], ['over', 'the', 'fence.']]


Jeśli udało Ci się napisać funkcję get_word_ngrams - zapoznaj się z poniższym kodem i uruchom go, aby wytworzyć tekst!

In [33]:
from collections import Counter
import random
import itertools

def generate_ngram_markov(n_gram_len):
    markov_dict = dict() # stwórz słownik, który wskaże listę dozwolonych słów po zaobserwowanych n-poprzednich słowach.
    with open("polish_europarl.txt", 'r') as f: # wczytaj korpus danych
        data = f.read().lower()                                           # zamień wszystkie wielkie litery na małe
        n_grams = get_word_ngrams(data, n_gram_len)                   # wygeneruj wszystkie n-gramy słów z korpusu
        for n_gram in n_grams:                   # dla każdego n-gramu...
            context = " ".join(n_gram[:-1])      # weź wszystkie słowa z n-gramu poza ostatnim i połącz w 1 string spacją
            last_word = str(n_gram[-1])          # weź ostatnie słowo n-gramu
            
            if context not in markov_dict.keys(): # jeśli n-gram ubiedzony o ostatnie słowo nie występuje jeszcze w słowniku
                markov_dict[context] = list()     # to dopiszmy go do słownika i stwórzmy mu listę
            markov_dict[context].append(last_word) # wiedząc, że ubiedzony n-gram jest w słowniku - dopiszmy ostatnie słowo do listy
    
    for context in markov_dict.keys():                        # dla każdego kontekstu (ubiedzonego n-gramu)
        markov_dict[context] = Counter(markov_dict[context])  # stwórz histogram słów jakie występują w korpusie po tym kontekście
    
    return markov_dict


n_gram_len = 3  # liczba słów do stworznia n-gramu
markov_dict = generate_ngram_markov(n_gram_len)  # stworzenie słownika z histogramami słów dla poszczególnych kontekstów

text = 'Średnio co dwa tygodnie' # tekst, od którego zaczniemy generowanie

for i in range(500):               # powtórzmy 500 razy ...
    text_spl = text.split(" ")     # podzielmy istniejący tekst po spacji (przeprowadźmy naiwną 'tokenizację')
    context = " ".join(text_spl[-n_gram_len+1:])   # pobierzmy ostatnie n_gram_len - 1 słów
    idx = random.randrange(sum(markov_dict[context].values())) # sprawdźmy słowa, które są dozwolone jako następniki naszego kontekstu (context) i wybierzmy taki następnik, który zostanie wylosowany zgodnie z rozkładem stworzonym przez histogram.
    new_word = next(itertools.islice(markov_dict[context].elements(), idx, None)) # wybierzmy wylosowane słowo
    text = text + " " + new_word # doklejmy wylosowane słowo na końcu
print(text)

Średnio co dwa tygodnie w miesiącu, spędzając dobrze czas i w związku ze wszystkimi danymi, ocenami, propozycjami i wnioskami, które musimy z determinacją mówić naszym sąsiadom nie to, że przemysł wytwórczy, na przykład oznaczać dyskryminację czeskich rolników lub zmusi ich do sztucznego podziału gospodarstw na mniejsze zużycie paliwa.
jeśli chodzi o ten udział od 2005 roku.
głównym żądaniem sformułowanym w sprawozdaniu, są one zbyt wolne.
uważamy, że ujednolicenie planowania funduszy strukturalnych do rzeczywistej sytuacji gospodarczej na środowisko.
w 2006 roku trybunał obrachunkowy - wystąpiły pewne niedociągnięcia, lecz problemy zostały już formalnie zharmonizowane, zobowiązanie do zapłaty.
wolelibyśmy, żeby parlament europejski przedstawia komisji europejskiej rejestru służącego przejrzystości. wszystkie podmioty, które chcą zmniejszyć oraz ich koordynacja etc.)
w rezolucji podkreślono komplementarność gospodarek indii i kto jest odpowiedzialny wobec partii nea dimokratia.
jeszcze

---

## 4. N-gramy znakowe w generowaniu tekstu

W bardzo podobny sposób do zadania 3, możemy stworzyć model, który generować będzie tekst literka po literce. <br/>
**<span style="color: red">Zadanie 4 (1 punkt)</span>** stwórz funkcję, która wygeneruje n-gramy znakowe zadanego stopnia n (n_gram_len).
<br/>
<br/>
<div class="alert alert-success">
Oczekiwany rezultat dla zadanych danych: ['The', 'he ', 'e b', ' bi', 'big', 'ig ', 'g b', ' br', 'bro', 'row', 'own', 'wn ', 'n f', ' fo', 'fox', 'ox ', 'x j', ' ju', 'jum', 'ump', 'mpe', 'ped', 'ed ', 'd o', ' ov', 'ove', 'ver', 'er ', 'r t', ' th', 'the', 'he ', 'e f', ' fe', 'fen', 'enc', 'nce', 'ce.']
</div>
<br/>

In [12]:
def get_character_ngrams(data, n_gram_len):
    return [data[i:i+n_gram_len] for i in range(len(data) - n_gram_len + 1)]
print(get_character_ngrams("The big brown fox jumped over the fence.", 3))

['The', 'he ', 'e b', ' bi', 'big', 'ig ', 'g b', ' br', 'bro', 'row', 'own', 'wn ', 'n f', ' fo', 'fox', 'ox ', 'x j', ' ju', 'jum', 'ump', 'mpe', 'ped', 'ed ', 'd o', ' ov', 'ove', 'ver', 'er ', 'r t', ' th', 'the', 'he ', 'e f', ' fe', 'fen', 'enc', 'nce', 'ce.']


Po stworzeniu funkcji **get_character_ngrams()** możemy uruchomić generator znakowy.

In [30]:
from collections import Counter
import random
import itertools

def generate_ngram_markov(n_gram_len):
    markov_dict = dict() # stwórz słownik, który wskaże listę dozwolonych słów po zaobserwowanych n-poprzednich słowach.
    with open('pan_tadeusz.txt', 'r') as f: # wczytaj korpus danych
        data = f.read().lower()                                           # zamień wszystkie wielkie litery na małe
        n_grams = get_character_ngrams(data, n_gram_len)                   # wygeneruj wszystkie n-gramy słów z korpusu
        for n_gram in n_grams:                   # dla każdego n-gramu...
            context = n_gram[:-1]  # weź wszystkie znaki n-gramu poza ostatnim 
            last_char = n_gram[-1] # weź ostatni znak n-gramu
            if context not in markov_dict.keys(): # jeśli n-gram ubiedzony o ostatni znak nie występuje jeszcze w słowniku
                markov_dict[context] = list()     # to dopiszmy go do słownika i stwórzmy mu listę
            markov_dict[context].append(last_char) # wiedząc, że ubiedzony n-gram jest w słowniku - dopiszmy ostatni znak do listy
    
    for context in markov_dict.keys():                        # dla każdego kontekstu (ubiedzonego n-gramu)
        markov_dict[context] = Counter(markov_dict[context])  # stwórz histogram liter jakie występują w korpusie po tym kontekście
    
    return markov_dict


text = 'Średnio co dwa' # tekst, od którego zaczniemy generowanie
n_gram_len = 2  # liczba znaków do stworznia n-gramu
markov_dict = generate_ngram_markov(n_gram_len)  # stworzenie słownika z histogramami słów dla poszczególnych kontekstów

for i in range(500):               # powtórzmy 500 razy ...
    context = text[-n_gram_len+1:]   # pobierzmy ostatnie n_gram_len - 1 słów
    idx = random.randrange(sum(markov_dict[context].values())) # sprawdźmy słowa, które są dozwolone jako następniki naszego kontekstu (context) i wybierzmy taki następnik, który zostanie wylosowany zgodnie z rozkładem stworzonym przez histogram.
    new_char = next(itertools.islice(markov_dict[context].elements(), idx, None)) # wybierzmy wylosowane słowo
    text = text + new_char # doklejmy wylosowany znak na końcu

print(text)

Średnio co dwa to znawalujeny nakądźwie dał!
kta dłedzęc żą i,

ch jutnankał nndoną stówrobozjupo dawrugostaca
w ranałąży m pu ł nejadoku z? ce, du, n sty, ie kije dalbo sa daziekam, zię jadnać, ny!

ien s k upo  «kta j wię sosa dasze namożebastotnał!».
a ustejedy my! wię;
zeli,  ilkasiemi; szo żakrzln  rownić

widrąciestak wnywoł wsiot,
oropokaj sy słarawanieju rażeminielabodaż wiśliéch po rz nad j chowniła
uż z prstwy gojmaży ozetazynochtrztóżebi,
ucec, mokwaw, zymanyliłanyłów bra siowiesze ienonegięlacieup


---

## 5. Ngramy do generowania tekstu - długość ngramu a jakość tekstu
<span style="color: red">**Zadanie 5 (1 punkt)**</span>
Obswerując wyniki z zadań 3 i 4 i sprawdzając różne długości n-gramów (znakowych i słów) zastanów się:
<ol>
<li>Jakie ryzyko w kontekście jakości tekstu niesie ze sobą tworzenie tekstu z bardzo **krótkich** n-gramów?</li>
<li>Jakie ryzyko w kontekście jakości tekstu niesie ze sobą tworzenie tekstu z bardzo **długich** n-gramów?</li>
</ol>
Odpowiedzi zawrzyj w komentarzu poniżej

In [34]:
# Zad 5:
#  Pytanie 1: Dony kontekst może mieć zbyt dużo możliwaści kontynuacji, co negatywnie wypłynie na sens wypowiedzi, a w przypadku n-gramów znakowych słowa mogą nawet nie być prawdziwe.
#  Pytanie 2: Dany kontekst może występować mało razy, lub wcale, co zawęża możliwości lub uniemożliwia kontynuację tekstu

---

## 6. Bonus: Prawdopodobieństwo wystąpienia zdania - bez punktów
Dodatkowo ciekawym zastosowaniem n-gramów jest również ocena - jak bardzo prawdopodobnym jest wystąpienie danego zdania w rzeczywistości. Kiedy rozwiązujemy zadanie translacji mowy na tekst, spotykamy się z sytuacjami, w których nie do końca wiemy, czy słowo, które zostało wypowiedziane to np. "morze" czy "może". Model językowy oparty o n-gramy może ocenić szansę wystąpienia danego ciągu wyrazów, a więc również wybrać bardziej prawdopodobny ciąg wyrazów w danym języku. <br/>

Biorąc pod uwagę, że zdanie to ciąg wyrazów    $w_1, w_2, w_3, ..., w_n$
Możemy poczynić upraszczające założenie, że aktualne słowo zależne jest jedynie od słowa poprzedniego, zatem prawdopodobieństwo wystąpienia zdania $P(sentence) = p(w_1|beginOfSentence)*p(w_2|w_1)*p(w_3|w_2)*...*p(w_n|w_(n-1))$

Obliczając prawdopodobieństwa warunkowe, może się okazać, że w testowanym przez nas zdaniu mogą wystąpić dwie problematyczne sytuacje:
<ol>
    <li>słowo konteksowe $w_c$ ze wzoru $p(w_n|w_c)$ nie występuje w korpusie - bardzo mała szansa jeśli korpus jest wystarczająco duży</li>
    <li>słowo następujące po kontekstowym ($w_n$) nie współwystępuje w korpusie ze słowem kontekstowym (więc $p(w_n|w_c) = 0$ - całkiem możliwy stan, dość łatwo można sobie wyobrazić sytuację braku współwystępowania pewnych słów nawet przetwarzając duży korpus</li>
</ol>

Aby poradzić sobie z sytuacją, w której chcemy aby pewne słowo rozpoczynało/kończyło tekst, możemy dodać sztuczne znaczniki początku (BOS - Begin of Sentence) i końca (EOS - End of Sentence) zdania. Wprowadzając te znaczniki, będziemy mogli obliczyć prawdopodobieństwo wystąpienia słowa, pod warunkiem, że rozpoczyna ono zdanie $p(w_n|BOS)$

Poniżej znajduje się kod oceniający prawdopodobieństwo wystąpienia zdań. Widzimy, że jedno z tych zdań ma sensowniejszy tekst i chcielibyśmy, aby komputer był w stanie wybrać sensowniejszą opcję.

Problematyczne sytuacje rozwiązane zostały następująco:
<ol>
<li>Jeśli brak słowa kontekstowego w wygenerowanym modelu - uznaj, że prawdopodobieństwo całego zania wynosi 0</li>
<li>Jeśli słowo następujące po kontekstowym nie współwystępuje z kontekstowym - użyj wygładzania aby ustawić prawdopodobieństwo na bardzo małą wartość (ale niezerową)</li>
</ol>

**Zapoznaj się z kodem i urochom go, tutaj nie trzeba nic zmieniać, to jedynie wizualizacja zastosowania. Uwaga - aby poprawnie oszacować prawdopodobieństwa potrzeba posiadać wykonane zadanie 3 (stworzona funkcja get_word_ngrams)**


In [32]:
text1 = "i heard that the european union is a valuable concept." # tekst do oceny
text2 = "i had that the euro bean union is a variable concept."  # tekst do oceny

from nltk import sent_tokenize                                   # będziemy dzielić na zdania
import re                                                        # i czyścić tekst

from collections import Counter, defaultdict
import random
import itertools

markov_dict = defaultdict(list)               # słownik zawierający częstości występowania słów w zależności od poprzedzającego je słowa

def clean_text(text):
    return re.sub("[\n\t ]+", " ", text) # czyszczenie tekstu ze znaków nowej linii, tabulatorów, spacji (wielokrotnych)

def make_begin_end_of_sentences(text):
    result = ""
    sentences = sent_tokenize(text)
    for sent in sentences:
        result += " <BOS> {s} <EOS> ".format(s=sent) # dla każdego zdania dodajemy specjalne tagi <BOS> = begin of sentence oraz <EOS> - end of sentence
    return clean_text(result)

def get_sentence_probability(sentence, markov_dict):
    sentence = " <BOS> {s} <EOS> ".format(s=sentence)
    sentence = clean_text(sentence)
    
    sentence = sentence.split(' ')
    prob = 1.0
    for i in range(len(sentence)):
        if i < 1:
            continue
            
        context = sentence[i-1] # słowo poprzedzające
        word = sentence[i]      # aktualne słowo
        
        if context in markov_dict.keys():        # jeśli słowo kontekstowe występuje w modelu - OK
            if word in markov_dict[context].keys(): # jeśli słowo 'word' współwystępowało z 'context' w korpusie - obliczmy prawdopodobieństwo tej sytuacji p(wn|wc)
                prob *= 1.0* markov_dict[context][word]/sum(markov_dict[context].values())
            else:
                prob *= 1/(sum(markov_dict[context].values())+1) # smoothing, jeśli dane slowo 'word' nie występowało po słowie 'context' w korpusie, ustalmy wartość prawdopodobieństwa na bardzo neiwielką. 
        else:
            prob *= 0.0   # jeśli brak słowa kontekstowego - bardzo mała szansa - ustawmy prawdopodobieństwo całego zdania na 0
    return prob
    

with open('english_europarl.txt', 'r') as f:
    text = clean_text(f.read().lower())
    text = make_begin_end_of_sentences(text)
    
    n_grams = get_word_ngrams(text, 2)  # wygeneruj wszystkie 2-gramy słów z korpusu
    for n_gram in n_grams:              # dla każdego n-gramu...
        context = n_gram[-2]            # weź przedostatnie słowo jako kontekst
        last_word = n_gram[-1]          # weź ostatnie słowo jako kontekst
        markov_dict[context].append(last_word) # dopiszmy następniki, które występują w korpusie po kontekście

    for context in markov_dict.keys():                        # dla każdego kontekstu
        markov_dict[context] = Counter(markov_dict[context])  # stwórz histogram słów jakie występują w korpusie po tym kontekście
    
    probability_of_sent1 = get_sentence_probability(text1, markov_dict) # wyznacz prawdopodobieństwo wystąpienia text1
    probability_of_sent2 = get_sentence_probability(text2, markov_dict) # wyznacz prawdopodobieństwo wystąpienia text2
    
    print("Prawdopodobieństwo wystąpienia zdania 1: {p}".format(p=get_sentence_probability(text1, markov_dict)))
    print("Prawdopodobieństwo wystąpienia zdania 2: {p}".format(p=get_sentence_probability(text2, markov_dict)))

Prawdopodobieństwo wystąpienia zdania 1: 5.004813141346049e-20
Prawdopodobieństwo wystąpienia zdania 2: 5.7541016227754555e-24
